In [1]:
import os
import importlib
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, Trainer, TrainingArguments, TrainerCallback, EarlyStoppingCallback
import torch
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import json

import prompts
import evaluate


/home/explorer/anaconda3/envs/lasse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL = 'llama'
DATA = 'acl_arc' #acl_arc, finecite, multicite
SCHEMA = 'XML' # XML, JSON
DATA_SIZE = None # 1000


INPUT = f'./data/{DATA}/{SCHEMA}/'
OUTPUT = f'./output/{DATA}/{MODEL}/{SCHEMA}/'
if DATA_SIZE:
    INPUT = f'./data/{DATA}/{DATA_SIZE}/{SCHEMA}/'
    OUTPUT = f'./output/{DATA}/{MODEL}/{DATA_SIZE}/{SCHEMA}/'

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_mapping = {
    "mistral":'mistralai/Mistral-7B-Instruct-v0.3',
    "scitulu":'allenai/scitulu-7b',
    'llama':'meta-llama/Meta-Llama-3.1-8B-Instruct',
}
model_id = model_mapping[MODEL]

max_seq_length = 1024

#tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

In [3]:
import importlib
importlib.reload(evaluate)

# #get prompt len
# for DATA in ['acl_arc', 'finecite', 'multicite']:
#     for SCHEMA in ['XML', 'JSON']:
#         prompt = prompts.PromptForAutoCCA(tokenizer, DATA, SCHEMA)
#         print(len(prompt.create_sample('','',return_prompt_only=True)))
            

#get len of tokenized train_data
# max_len = 0
# for ids in train_ds['input_ids']:
#     if len(ids) > max_len: max_len = len(ids)  
# print(max_len)

<module 'evaluate' from '/home/explorer/Automatic_CCE/evaluate.py'>

In [4]:
#model
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)

LMmodel = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config = bnb_config,
    torch_dtype = torch.bfloat16,
    device_map ='auto'
)
LMmodel.resize_token_embeddings(len(tokenizer))

peft_config = LoraConfig(
        target_modules=[ "v_proj", "q_proj", "up_proj", "o_proj", "k_proj", "down_proj", "gate_proj" ], 
        inference_mode=False, 
        r=4, 
        lora_alpha=32, 
        lora_dropout=0.1,    
        task_type="CAUSAL_LM",        
    )

LMmodel = get_peft_model(LMmodel, peft_config)

LMmodel.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.62s/it]


trainable params: 10,485,760 || all params: 8,040,747,008 || trainable%: 0.1304


In [5]:
# load data
with open(INPUT + 'train.json', 'r') as file:
    train_data = json.load(file)
    
with open(INPUT + 'test.json', 'r') as file:
    test_data = json.load(file)

# Convert the DataFrame to a Dataset
train_ds = Dataset.from_list(train_data)
test_ds = Dataset.from_list(test_data[:int(len(test_data)/2)])

# initialize prompt class

prompt = prompts.PromptForAutoCCA(tokenizer, MODEL, DATA, SCHEMA)

#Apply the tokenization function to the dataset
train_ds = train_ds.map(
    lambda row: prompt.create_sample(row['input'], row['output']), 
    batched=False, 
    remove_columns=train_ds.column_names
)

dev_ds = test_ds.map(
    lambda row: prompt.create_sample(row['input'], row['output']), 
    batched=False, 
    remove_columns=test_ds.column_names
)

eval_ds = test_ds.map(
    lambda row: prompt.create_sample(row['input'], row['output'], for_generation=True), 
    batched=False, 
)


Map: 100%|██████████| 867/867 [00:00<00:00, 1087.42 examples/s]


In [6]:
# max_len = 0
# for ids in train_ds['input_ids']:
#     if len(ids) > max_len: max_len = len(ids)
# max_len

In [7]:
# Define Data Collator
class CustomDataCollator:
    def __init__(self, tokenizer, model_id, padding, max_length):
        self.tokenizer = tokenizer
        self.model_id = model_id
        self.padding = padding
        self.max_length = max_length

    def __call__(self, features):
        batch = self.tokenizer.pad(
            features,
            padding=self.padding,
            max_length=self.max_length,
            return_tensors='pt',
        )
        labels = batch["input_ids"].clone()
        
        # Compute loss mask for output token only
        for i in range(batch['input_ids'].shape[0]):
            
            # Decode whole sample
            text_content = self.tokenizer.decode(batch['input_ids'][i][1:])  
            
            # Extract output boundary
            if self.model_id == 'mistral':
                output_boundary = text_content.rfind("[/INST]") + len("[/INST]")
            if self.model_id == 'llama':
                output_boundary = text_content.rfind("<|end_header_id|>") + len("<|end_header_id|>")
            prompt_text = text_content[:output_boundary]
            
            # tokenize promt text
            prompt_text_tokenized = self.tokenizer(
                prompt_text,
                return_overflowing_tokens=False,
                return_length=False,
            )
            # get length of promt text
            promt_text_len = len(prompt_text_tokenized['input_ids'])
            
            # set loss mask for promt text
            labels[i][range(promt_text_len)] = -100
            
                    
        batch["labels"] = labels
        return batch

# init data collator
data_collator=CustomDataCollator(
    tokenizer=tokenizer, 
    model_id = MODEL,
    padding="longest", 
    max_length=max_seq_length, 
)

In [8]:
# from sklearn.metrics import (accuracy_score, 
#                              recall_score, 
#                              precision_score, 
#                              f1_score)

# def compute_metrics(p):    
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)
#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     recall = recall_score(y_true=labels, y_pred=pred)
#     precision = precision_score(y_true=labels, y_pred=pred)
#     f1 = f1_score(y_true=labels, y_pred=pred)    
#     return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [9]:
# hyper parameterh
params =  {
    'lr': 1e-5
}

# load trainer
training_arguments = TrainingArguments(
    output_dir=OUTPUT,
    eval_strategy = 'steps',
    eval_steps= 500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=params['lr'],
    #num_train_epochs = 9,
    max_steps=len(train_data) * 3,
    warmup_ratio=0.1,
    weight_decay=0.01,
    logging_steps = 100,
    save_strategy = 'steps',
    save_steps=500,
    save_total_limit=3,
    load_best_model_at_end=True,
    label_names = ['labels'],
)
trainer = Trainer(
    model=LMmodel,
    train_dataset=train_ds,
    eval_dataset=dev_ds,
    tokenizer=tokenizer,
    args=training_arguments,
    data_collator=data_collator,
    callbacks= [EarlyStoppingCallback(early_stopping_patience=2)]
)

# par? 2500 steps wiht 1e-04? SFTTrainer? arr as answer

In [10]:
trainer.train()

{'loss': 0.1823, 'grad_norm': 2.4405770301818848, 'learning_rate': 4.5146726862302486e-07, 'epoch': 0.0135464643727987}
{'loss': 0.17, 'grad_norm': 3.369896411895752, 'learning_rate': 9.029345372460497e-07, 'epoch': 0.0270929287455974}
{'loss': 0.1161, 'grad_norm': 3.758136510848999, 'learning_rate': 1.3544018058690746e-06, 'epoch': 0.0406393931183961}
{'loss': 0.0645, 'grad_norm': 3.7456984519958496, 'learning_rate': 1.8058690744920994e-06, 'epoch': 0.0541858574911948}
{'loss': 0.0473, 'grad_norm': 3.3516476154327393, 'learning_rate': 2.2573363431151243e-06, 'epoch': 0.0677323218639935}
{'eval_loss': 0.04754631593823433, 'eval_runtime': 252.7716, 'eval_samples_per_second': 3.43, 'eval_steps_per_second': 3.43, 'epoch': 0.0677323218639935}
{'loss': 0.038, 'grad_norm': 1.7266865968704224, 'learning_rate': 2.7088036117381493e-06, 'epoch': 0.0812787862367922}
{'loss': 0.0352, 'grad_norm': 3.158332586288452, 'learning_rate': 3.1602708803611743e-06, 'epoch': 0.09482525060959089}
{'loss': 0.0

KeyboardInterrupt: 

In [11]:
import importlib
importlib.reload(evaluate)

evaluator = evaluate.Evaluator(trainer.model, MODEL, tokenizer, eval_ds, DATA, SCHEMA, DEVICE)
results = evaluator.evaluate() #test_data=eval_ds['output'])

results['params'] = params
results['notes'] = ''

with open(OUTPUT + 'eval_results.jsonl', 'a') as fp:
    json.dump(results, fp)

  1%|          | 1/150 [00:11<28:58, 11.67s/it]

This paper considers the reading comprehension task in which some discrete-reasoning abilities are needed to correctly answer questions. <CONT> Specifically, we focus on a new reading comprehension dataset called DROP #TARGET_REF<USE/>, which requires Discrete Reasoning Over the content of Paragraphs to obtain the final answer. </CONT> Unlike previous benchmarks such as CNN/DM (#REF) and SQuAD (#REF) that have been well solved #REF), DROP is substantially more challenging in three ways. First, the answers to 1 https://github.com/huminghao16/MTMSN the questions involve a wide range of types such as numbers, dates, or text strings. Therefore, various kinds of prediction strategies are required to successfully find the answers.


  1%|▏         | 2/150 [00:24<29:53, 12.12s/it]

Based on the representations, we employ a multi-type answer predictor that is able to produce four answer types: (1) span from the text; (2) arithmetic expression; (3) count number; (4) negation on numbers ( §3.2). <CONT> Following #TARGET_REF<USE/>, we first predict the answer type of a given passage-question pair, and then adopt individual prediction strategies. </CONT> To support multispan extraction ( §3.3), the model explicitly predicts the number of answer spans. It then outputs non-overlapped spans until the specific amount is reached. Moreover, we do not directly use the arithmetic expression that possesses the maximum probability, but instead re-rank several expression candidates that are decoded by beam search to further confirm the prediction ( §3.4).


  2%|▏         | 3/150 [00:38<31:56, 13.04s/it]

...

 99%|█████████▉| 149/150 [29:54<00:11, 11.04s/it]

<CONT> FAIRSEQ supports language modeling with gated convolutional models and Transformer models #TARGET_REF<BACKGROUND/>. </CONT> Models can be trained using a variety of input and output representations, such as standard token embeddings, convolutional character embeddings (Kim 1 En-De En-Fr a. #REF 25.2 40.5 b. #REF 28.4 41.0 c. #REF 28.9 41.4 d. #REF 29 et al., 2016), adaptive softmax (#REF), and adaptive inputs. We also provide tutorials and pre-trained models that replicate the results of and


100%|██████████| 150/150 [30:04<00:00, 12.03s/it]

FAIRSEQ supports language modeling with gated convolutional models and Transformer models (#REF). <CONT> Models can be trained using a variety of input and output representations, such as standard token embeddings, convolutional character embeddings (Kim 1 En-De En-Fr a. #REF 25.2 40.5 b. #TARGET_REF<BACKGROUND/> 28.4 41.0 c. #REF 28.9 41.4 d. #REF 29 et al., 2016), adaptive softmax (#REF), and adaptive inputs. </CONT> We also provide tutorials and pre-trained models that replicate the results of and
Structure failed at function: is_similar_to_input, {'pattern': '<(?:BACKGROUND|USE|DIFFERENCE|SIMILARITY|MOTIVATION|EXTENDS|FUTURE)/>|</?CONT>'}
{'opening': [], 'closing': [], 'self_closing': ['<USE/>']}
Since DROP does not indicate the answer type but only provides the answer string, we therefore adopt the weakly supervised annotation scheme, as suggested in #REF; #TARGET_REF<USE/>. We find all possible annotations that point to the gold answer, including matching spans, arithmetic expres

Structure failed at function: is_similar_to_input, {'pattern': '<(?:BACKGROUND|USE|DIFFERENCE|SIMILARITY|MOTIVATION|EXTENDS|FUTURE)/>|</?CONT>'}
Structure failed at function: is_similar_to_input, {'pattern': '<(?:BACKGROUND|USE|DIFFERENCE|SIMILARITY|MOTIVATION|EXTENDS|FUTURE)/>|</?CONT>'}
{'opening': [], 'closing': [], 'self_closing': ['<USE/>']}
(26.5% error reduction). Evaluation on the CRAC data set 3 For the CRAC data set, we train the #REF system end-to-end on the reduced corpus with singleton mentions removed and extract mentions from the system by set λ = 0.4. We then train our models with the same λ but on the full corpus, since our mention detectors naturally support both mention 3 As the #TARGET_REF<USE/> system does not predict singleton mentions, the results on CRAC data set in Table 2 are evaluated without singleton mentions. While the results reported in Table 3 are evaluated with singleton mentions included. 88.0 89.7 89.1 Table 3 : Comparison between our BIAFFINE MD and